# ResNet в Keras

Разберём, как реализована архитектура ResNet в Keras.

Импортируем ResNet из Keras. 50 означает количество слоёв в сети.

```
from tensorflow.keras.applications.resnet import ResNet50

model = ResNet50(input_shape=None,
                 classes=1000,
                 include_top=True,
                 weights='imagenet')
```

Рассмотрим все аргументы:

- `input_shape` — размер входного изображения. Например: `(640, 480, 3)`.
- `classes=1000` — количество нейронов в последнем полносвязном слое, в котором выполняется классификация.
- `weights='imagenet'` (от англ. «сеть изображений») — инициализация весов. ImageNet — название большого датасета, на котором сеть обучалась классифицировать изображения на 1000 классов. Если обучение сети начать на ImageNet, а продолжить на вашей задаче, результат будет лучше, чем если обучать с нуля. Чтобы инициализация весов была случайной, напишите `weights=None`.
- `include_top=True` (англ. «добавить верхушку») — указание на то, что в конце архитектуры ResNet есть два слоя: GlobalAveragePooling2D и Dense. Если задать False, то этих слоёв не будет.

Рассмотрим последние слои:
- GlobalAveragePooling2D (англ. «глобальный двумерный пулинг усреднением») — пулинг с окном во весь тензор. Как и AveragePooling2D, возвращает среднее значение из группы пикселей внутри канала. GlobalAveragePooling2D нужен, чтобы усреднить информацию по всему изображению, то есть получить пиксель с большим количеством каналов (например, 512 для ResNet50).
- Dense — полносвязный слой для классификации.

Разберём, как применять предобученную на ImageNet сеть. Чтобы адаптировать ResNet50 к нашей задаче, уберём верхушку и сконструируем её заново:
```
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential

backbone = ResNet50(input_shape=(150, 150, 3),
                    weights='imagenet', 
                    include_top=False)

model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(12, activation='softmax'))
```

где `backbone` (англ. «костяк») — то, что осталось от ResNet50.

Научим вас трюку. Допустим, есть очень маленький датасет: всего 100 картинок и два класса. Если на таком датасете обучить ResNet50, то она гарантированно переобучится: в ней слишком много параметров — порядка 23 млн! У сети будут идеальные предсказания на обучающей выборке и случайные — на тестовой.

Чтобы этого избежать, «заморозим» часть сети: некоторые слои оставим с весами из ImageNet, они не будут обучаться градиентным спуском. Обучим только 1–2 полносвязных слоя наверху сети. Так количество параметров в сети уменьшится, но архитектура сохранится.

Заморозим сеть так:
```
backbone = ResNet50(input_shape=(150, 150, 3),
                    weights='imagenet', 
                    include_top=False)

# замораживаем ResNet50 без верхушки
backbone.trainable = False

model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(12, activation='softmax'))
```

Чтобы сеть обучалась, добавленный сверху backbone полносвязный слой замораживать не стали.

Заморозка позволяет избавиться от переобучения и повысить скорость обучения сети:  градиентному спуску считать производные для замороженных слоёв не нужно.

Чтобы код выполнялся быстрее, мы загрузили веса модели ResNet50 на сервер. Скачивать веса сервер не будет, если в аргументе weights указать такой путь к файлу:

```
backbone = ResNet50(input_shape=(150, 150, 3),
                    weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                    include_top=False)
```

Сначала ваш код должен пройти предварительную проверку, а затем его поставят в очередь на обучение. В это время вы можете вернуться в основной курс и продолжить изучать уроки о компьютерном зрении. Когда пройдёт 2–3 часа, загляните в этот урок и проверьте, не завершилось ли обучение модели. Перейти к следующей задаче вы сможете только после того, как решите эту.

In [1]:
# Подключение библиотеки
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Загрузка обучающей выборки
def load_train(path):
    train_datagen = ImageDataGenerator(
        validation_split=0.25, 
        rescale=1./255, 
        horizontal_flip=True
    )

    
    train_datagen_flow = train_datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        subset='training',
        seed=12345
    )
    
    return train_datagen_flow

In [3]:
# Создание модели
def create_model(input_shape):
    # Инициализируем модель машинного обучения
    model = Sequential()
    # Добавляем модель ResNet50
    model.add(ResNet50(input_shape=input_shape, classes=1000, include_top=False, weights='imagenet'))
    
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(300, activation='relu'))
    
    model.add(Dense(12, activation='softmax'))
    
    # Для настройки гиперпараметров: Основной настраиваемый гиперпараметр в алгоритме Adam — скорость обучения (learning rate). 
    # Это шаг градиентного спуска, с которого алгоритм стартует.
    # По умолчанию он равен 0.001. Уменьшение шага иногда может замедлить обучение, но улучшить итоговое качество модели.
    optimizer_adam = Adam(lr=0.00001)
    
    # Также устанавливаем параметры, отвечающие за обучение
    model.compile(optimizer=optimizer_adam, loss='sparse_categorical_crossentropy', metrics=['acc'])
    
    return model

In [4]:
# Запуск модели
def train_model(model, train_data, test_data, batch_size=None, epochs=10, steps_per_epoch=None, validation_steps=None):      
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
 
    return model

# Вывод на консольное окно